In [1]:
# import the neccessary libraries

import pandas as pd
import numpy as np
from pulp import  *

In [2]:
# import the dataset into a dataframe

df = pd.read_csv("fau_bank_shifts.csv", index_col=0)

df = df.fillna(0).applymap(lambda x: 1 if x == "X" else x)

C:\Users\prabh\AppData\Local\Temp\ipykernel_14620\1823470405.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.fillna(0).applymap(lambda x: 1 if x == "X" else x)


In [3]:
# read the avg number of customers for each day of the week in an array

avg_customer_num_columns = ["Avg_Customer_Number"]

In [4]:
# create a matrix to show which shift each time window is associated with

shifts = df.drop(index=["Wage rate per 4h shift (EUR)"], columns=avg_customer_num_columns).values

In [5]:
print(shifts)

[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]


In [6]:
# read the rest of the data from the the csv file

# number of shifts
shift_num = shifts.shape[1]

# number of time windows
time_windows = shifts.shape[0]

# number of customers measured per time window
avg_customer_num = df[avg_customer_num_columns].values

# wage rate per shift
wages_per_shift = df.loc["Wage rate per 4h shift (EUR)", :].values.astype(int)

# service level
service_level = np.array([[0.125, 0, 0], [0, 0, 0]]).astype(float)[0]

In [7]:
print("Number of shifts: ", shift_num)
print("Number of time windows: ", time_windows)
print("Number of customers measured per time window: ", avg_customer_num)
print("Wage rate per shift: ", wages_per_shift)
print("Service level: ", service_level)

Number of shifts:  2
Number of time windows:  8
Number of customers measured per time window:  [[28.]
 [35.]
 [21.]
 [46.]
 [32.]
 [14.]
 [24.]
 [32.]
 [ 0.]]
Wage rate per shift:  [16 14  0]
Service level:  [0.125 0.    0.   ]


In [8]:
# Determine the decision variable
# final goal is to find the optimal number of workers for each time slot of each day

num_workers_indexes = []
for day_of_week in range(0,1):
  for shift_index in range(shift_num):
    num_workers_indexes.append(f'{day_of_week}_{shift_index}')
    
num_workers = LpVariable.dicts("num_workers", num_workers_indexes, lowBound=0, cat="Integer")
print(num_workers)

{'0_0': num_workers_0_0, '0_1': num_workers_0_1}


In [9]:
# Create problem
# Minimize number of workers/costs paid for employees each day

prob = LpProblem("scheduling_workers", LpMinimize)

In [10]:
# add the contsraints to the LP problem
# first add to the problem the wage for each shift

prob += lpSum([[wages_per_shift[j] * num_workers[f'{day_of_week}_{j}'] for j in range(shift_num)]] for day_of_week in range(0,1))

In [11]:
print(prob)

scheduling_workers:
MINIMIZE
16*num_workers_0_0 + 14*num_workers_0_1 + 0
VARIABLES
0 <= num_workers_0_0 Integer
0 <= num_workers_0_1 Integer



In [12]:
# the average number of customers in each time slot and the service level are also added to the LP problem

print(avg_customer_num)
for day_of_week in range(0, 1):
  for t in range(time_windows):
    print(avg_customer_num[t][day_of_week])
    prob += lpSum([shifts[t, j] * num_workers[f'{day_of_week}_{j}'] for j in range(shift_num)]) >= avg_customer_num[t][day_of_week] * service_level

[[28.]
 [35.]
 [21.]
 [46.]
 [32.]
 [14.]
 [24.]
 [32.]
 [ 0.]]
28.0
35.0
21.0
46.0
32.0
14.0
24.0
32.0


In [13]:
# solve the problem

prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


In [14]:
# print the results

for index in num_workers:
  index_parts = index.split('_')
  day_of_week = int(index_parts[0])
  shift = index_parts[1]
  print(
    f"The number of workers needed for shift {shift} is {int(num_workers[index].value())} workers"
  )

The number of workers needed for shift 0 is 6 workers
The number of workers needed for shift 1 is 4 workers
